In [1]:
import pandas as pd
import numpy as np
from requests import get
from bs4 import BeautifulSoup
from time import sleep, time
from random import randint
from warnings import warn
from IPython.core.display import clear_output

In [2]:
def extrairPrimeiroNome(nomes = [], *args):
    primeiroNome = []
    for nome in nomes:
        primeiroNome.append(nome.partition(' ')[0])
    return primeiroNome


In [3]:
paginas = [str(i) for i in range(1,414)]

requests = 0
start_time = time()
elapsed_time = 0
nomes = []

for pagina in paginas:
    url_UFRN = 'http://www.portaldatransparencia.gov.br/servidores/'\
        'OrgaoExercicio-ListaServidores.asp?CodOrg=26243&Pagina={}'.format(pagina)
    response = get(url_UFRN)
    
    sleep(randint(1,6))
    
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)
    
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
              
    if requests > 413:
        warn('Numero de requests maior que o espperado')  
        break
        
    pagina_soup = BeautifulSoup(response.text, 'html.parser')
    nomes_servidores = pagina_soup.select("div #listagem table tr td a")
    for nome_serv in nomes_servidores:
        nomes.append(nome_serv.text)


Request:413; Frequency: 0.2166237572882272 requests/s


In [12]:
dfNomes = pd.DataFrame()
dfNomes['Nome do Servidor'] = nomes
dfNomes.to_csv('nomesSeridores.csv')

In [16]:
dfPrimeiroNome = pd.DataFrame()
dfPrimeiroNome['Nome'] = extrairPrimeiroNome(nomes)
dfPrimeiroNome.to_csv('primeiroNomeServidores.csv')